Visualización de los valores climáticos gatillo por provincia en España para ultimos años

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from IPython.display import display
import os

# **1. Cargar los DataFrames necesarios**

# Definir la ruta de la carpeta 'data'
data_dir = os.path.join('data')

# Cargar los valores gatillo desde el archivo CSV en la carpeta 'data'.
df_gatillo = pd.read_csv(os.path.join(data_dir, 'nuevos_valores_gatillo.csv'))

# Cargar el shapefile de las provincias de España desde la carpeta 'data'.
# Asegúrate de que todos los archivos del shapefile (.shp, .shx, .dbf, etc.) estén en la carpeta 'data'.
gdf_provincias = gpd.read_file(os.path.join(data_dir, "\georef-spain-provincia\georef-spain-provincia-millesime.shp"))

# **2. Preparar y unir los DataFrames**

# Renombrar la columna de provincia en 'gdf_provincias' para que coincida con 'df_gatillo'.
gdf_provincias.rename(columns={'prov_name': 'provincia'}, inplace=True)

# Normalizar los nombres de las provincias para evitar discrepancias al unir los DataFrames.
df_gatillo['provincia'] = df_gatillo['provincia'].str.upper().str.strip()
gdf_provincias['provincia'] = gdf_provincias['provincia'].str.upper().str.strip()

# Unir el DataFrame de valores gatillo con el GeoDataFrame de provincias.
gdf_filtrado = gdf_provincias.merge(df_gatillo, on='provincia', how='left')

# **3. Crear y personalizar el mapa**

# Crear un mapa centrado en España.
mapa = folium.Map(location=[40.4165, -3.7026], zoom_start=6)

# Añadir un mapa coroplético que colorea las provincias según la temperatura máxima gatillo.
folium.Choropleth(
    geo_data=gdf_filtrado,
    name='choropleth',
    data=gdf_filtrado,
    columns=['provincia', 'gatillo_temp_max_max'],
    key_on='feature.properties.provincia',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Temperatura Máxima Gatillo (°C)'
).add_to(mapa)

# Añadir las fronteras de las provincias y tooltips personalizados.
folium.GeoJson(
    gdf_filtrado,
    style_function=lambda x: {'fillColor': '#ffffff00', 'color': 'black', 'weight': 1},
    tooltip=folium.GeoJsonTooltip(
        fields=['provincia', 'gatillo_temp_max_max', 'gatillo_humedad_max'],
        aliases=['Provincia', 'Temp. Máx. Gatillo (°C)', 'Humedad Máx. Gatillo (%)'],
        localize=True,
        sticky=False,
        labels=True,
        toLocaleString=True,
        template="""
            <div style="font-size: 14px;">
                <b>{provincia}</b><br>
                Temp. Máx. Gatillo: {gatillo_temp_max_max:.2f}°C<br>
                Humedad Máx. Gatillo: {gatillo_humedad_max:.2f}%<br>
            </div>
        """
    )
).add_to(mapa)

# **4. Mostrar el mapa interactivo**

# Visualizar el mapa generado.
display(mapa)
